### PathMovers

This notebook is an introduction to handling `PathMover` and `PathMoveChange` instances. It mostly covers the questions on 
1. how to check if a certain mover was part of a change. 
2. What are the possible changes a mover can generate.
3. ...

Load OPENPATHSAMPLING

In [1]:
import openpathsampling as p

Let's open a file that contains a mover and lot's of changes

In [2]:
st = p.storage.Storage('_toy_retis.nc', mode='r')

A Simulator creates simulation steps. So we load a single step.

In [3]:
mc = st.steps[3]
print mc

Each step posesses a pathmovechange which we will get

In [4]:
pmc = mc.change
print pmc

PathSimulatorStep : PathSampling : Step # 3 with 0 samples
 +- RandomChoice :
     +- RandomChoice :
         +- SampleMove : PathReversalMover : False :[]


And the mover that was used to generate this change

In [5]:
pm = pmc.mover
print pm

PathSimulator


Let's first check if the pmc was really created by the pathmover

In [6]:
pmc in pm

True

This should be obvious, but under the hood there is some more fancy stuff happening. The `in` keyword with pathmovers and changes actually works on trees of movers. These trees are used to represent a specific order or movers being called and these trees are unique for changes. This means there is a way to label each possible pmc that can be generated by a mover. This possible only refers to choices in actual movers. 

This is due to special PathMovers that can pick one or more movers among a set of possibilities. The simplest of these is the OneWayShooter which choses equal between forward and backward shooting. So a OneWayShooter has two possible outcomes. Let's look at these

In [7]:
ow_mover = p.OneWayShootingMover([], []) # we use dummy arguments since are not going to use it

The property `enum` will evaluate the (potentially VERY large) number of all possible changes

In [8]:
ow_mover.enum

[(OneWayShooting, (BackwardShoot,)), (OneWayShooting, (ForwardShoot,))]

which is two in our case. Another example

In [9]:
ow_mover_2 = p.RandomChoiceMover([
        p.OneWayShootingMover([], []),
        p.OneWayShootingMover([], [])
        ])

In [10]:
ow_mover_2.enum

[(RandomChoice, (OneWayShooting, (ForwardShoot,))),
 (RandomChoice, (OneWayShooting, (BackwardShoot,))),
 (RandomChoice, (OneWayShooting, (BackwardShoot,))),
 (RandomChoice, (OneWayShooting, (ForwardShoot,)))]

Now we have 4 choices as expected 2 for the RandomChoice * 2 for the OneWayShooter. Although effectively there are only 2 distinct ones. Why is that? The problem in `ow_mover_2` is that we defined two separate instances of the OneWayShootingMover and in general different instances are considered different. In this case it might even be possible to check for equality, but we decided to leave this to the user. If you create two instances we assume there is a reason. Maybe just calling the differently (although it makes no sense for the monte carlo scheme).

The next example uses the same mover twice

In [11]:
ow_mover_3 = p.RandomChoiceMover([
        ow_mover,
        ow_mover
        ])

In [12]:
ow_mover_3.enum

[(RandomChoice, (OneWayShooting, (BackwardShoot,))),
 (RandomChoice, (OneWayShooting, (ForwardShoot,)))]

And we get only two distinct movers as we wanted to.

Let's look at the real mover

In [13]:
print len(pm.enum)
pm.enum[0:20:5]

31


[(PathSimulator,
  (RootMover,
   (Minus,
    (EnsembleFilter,
     (ConditionalSequential,
      (MinusSubtrajectoryChooser, (FirstSubtrajectorySelect,))))))),
 (PathSimulator,
  (RootMover,
   (Minus,
    (EnsembleFilter,
     (ConditionalSequential,
      (MinusSubtrajectoryChooser, (FirstSubtrajectorySelect,)),
      (InterfaceSetChooser,),
      (MinusExtensionDirectionChooser, (BackwardExtend,))))))),
 (PathSimulator, (RootMover, (ReplicaExchange, (ReplicaExchange,)))),
 (PathSimulator, (RootMover, (ReversalChooser, (PathReversal,))))]

This one has 31. Let's if we can reconstruct that number. The pathmover first choses between 4 different move types. 

1. Shooting, with one shooter for each of the 6 ensembles and two directions (forward and backward) are 6 * 2 = 12. 
2. Reversal one for each ensemble = 6
3. ReplicaExchange between all neighboring ensembles [1,2], [2,3], ..., [5,6]  = 5
4. And a Minus Move that consists of choices for picking the first or final inner trajectory from the minus ensemble and 2 choices for extending forward or backward. Here is important that we use a ConditionalSequentialMover that might call only some of its inner sequence. To be precise it could call only the subtrajectoryselection (2 choices), the selection and the replicaexchange (also 2 choices) or all three movers (4 choices). The will give a total of 8 possibilities

In total we have 12 + 6 + 5 + 8 = 31 as `enum` predicted.

Let's do some more testing

First let's check if the keys we get for all changes in the storage are actually contained in the mover or put more simple the mover could have generated these changes. This should be true for all changes that originate from our mover.

In [14]:
[pc in pm for pc in st.pathmovechanges[0:20]]

[False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 True]

What happened here? Why are only some changes made by our mover? Shouldn't all of them (except the 2 in the beginning) be the result of our pathmover? Yes and no. They are, but we are checking if the total change is made from a mover and we are also checking subchanges.

In [15]:
print st.pathmovechanges[2]
print st.pathmovechanges[2] in pm
print
print st.pathmovechanges[5]
print st.pathmovechanges[5] in pm


PathSimulatorStep : PathSampling : Step # 1 with 0 samples
 +- RandomChoice :
     +- RandomChoice :
         +- OneWayShooting :
             +- SampleMove : BackwardShootMover : False :[]
True

OneWayShooting :
 +- SampleMove : BackwardShootMover : False :[]
False


That seems bad, how to we get all the changes that are really important? One way is to scan all changes and use only the ones from the mover

In [16]:
real_changes = filter(lambda x : x in pm, st.pathmovechanges)
print len(real_changes), 'of', len(st.pathmovechanges)

200 of 1120


While this works it would be better to rely on the steps in the simulation and not on the changes itself. The steps point to the changes that are used to generate the next sampleset that we wanted. 

In [17]:
step_changes = [step.change for step in st.steps[1:]]
print len(step_changes)

200


We exclude the first step since it is the initialization which is not generated by our pathmover.

Let's now check which and how often one the 31 possible steps was called.

In [18]:
import collections

In [19]:
counter = collections.defaultdict(lambda: 0)

In [20]:
for ch in step_changes:
    counter[ch.unique] += 1

Let's make this better readable and define a recursive function that turn the tree of tuples into a nested string.

In [21]:
s = '%d of %d different changes run' % (len(counter), len(pm.enum))
print s, '\n', '-' * len(s), '\n'

for y in sorted(counter.items(), key=lambda x : -x[1]):
    print
    print y[1], 'x',  
    print p.treelogic.TupleTree.treeprint(y[0])

25 of 31 different changes run 
------------------------------ 


12 x PathSimulator
 +- RootMover
     +- ReplicaExchange
         +- ReplicaExchange

12 x PathSimulator
 +- RootMover
     +- ReversalChooser
         +- PathReversal

11 x PathSimulator
 +- RootMover
     +- ShootingChooser
         +- OneWayShootingMover I'face 1
             +- ForwardShoot

11 x PathSimulator
 +- RootMover
     +- ReversalChooser
         +- PathReversal

11 x PathSimulator
 +- RootMover
     +- ShootingChooser
         +- OneWayShootingMover I'face 1
             +- BackwardShoot

11 x PathSimulator
 +- RootMover
     +- ReversalChooser
         +- PathReversal

10 x PathSimulator
 +- RootMover
     +- ShootingChooser
         +- OneWayShootingMover I'face 3
             +- ForwardShoot

10 x PathSimulator
 +- RootMover
     +- ReplicaExchange
         +- ReplicaExchange

10 x PathSimulator
 +- RootMover
     +- ReversalChooser
         +- PathReversal

10 x PathSimulator
 +- RootMover
     +- Shoo

We see that especially Minus Moves are underrepresented. We should realize that all listed possibilies of the minus move can occur most of these are very rare since they require extremely long trajectories or a spontaneous cross to another state which is by construction very unlikely. If it were very likely we do not need OPS since we could sample transitions just by regular MD!

What else can we learn or ask for?

We can check if a particular mover was run. Let's pick the very last MinusExtensionDirectionChoser. To check for it we can either just check for the instance in a pmc

In [22]:
medc = pm[1][4][1][1][3]
print medc

MinusExtensionDirectionChooser


In [23]:
%%time
for pc in step_changes:
    if medc in pc:
        print repr(pc)
        print p.treelogic.TupleTree.treeprint(pc.unique)
        print

PathSimulator([215])
PathSimulator
 +- RootMover
     +- Minus
         +- EnsembleFilter
             +- ConditionalSequential
                 +- MinusSubtrajectoryChooser
                 |   +- FinalSubtrajectorySelect
                 +- InterfaceSetChooser
                 +- MinusExtensionDirectionChooser
                     +- BackwardExtend

PathSimulator([563])
PathSimulator
 +- RootMover
     +- Minus
         +- EnsembleFilter
             +- ConditionalSequential
                 +- MinusSubtrajectoryChooser
                 |   +- FirstSubtrajectorySelect
                 +- InterfaceSetChooser
                 +- MinusExtensionDirectionChooser
                     +- ForwardExtend

PathSimulator([785])
PathSimulator
 +- RootMover
     +- Minus
         +- EnsembleFilter
             +- ConditionalSequential
                 +- MinusSubtrajectoryChooser
                 |   +- FinalSubtrajectorySelect
                 +- InterfaceSetChooser
                 +- MinusExten

the expression `mover_instance in pmc` checks for the overall appearance of a mover somewhere independent of the location.

The second possibilty is to use a locator. 
A locator is a tupletree that is so specific that it will test for a specific mover. 

In [24]:
locators = pm.locators()

In [25]:
medc_locator = filter(lambda (key, mover) : mover is medc, locators.iteritems())[0][0]

In [26]:
print p.treelogic.TupleTree(medc_locator)

PathSimulator
 +- RootMover
     +- Minus
         +- EnsembleFilter
             +- ConditionalSequential
                 +- MinusSubtrajectoryChooser
                 +- InterfaceSetChooser
                 +- MinusExtensionDirectionChooser


In [27]:
pmc_tuple = p.treelogic.TupleTree(st.pathmovechanges[215].unique)
print medc_locator in pmc_tuple
print pmc_tuple

True
PathSimulator
 +- RootMover
     +- Minus
         +- EnsembleFilter
             +- ConditionalSequential
                 +- MinusSubtrajectoryChooser
                 |   +- FinalSubtrajectorySelect
                 +- InterfaceSetChooser
                 +- MinusExtensionDirectionChooser
                     +- BackwardExtend


This locator will test if the MinusExtentionDirectionChooser was called and only at this position. Imagine you have the same mover at another place in the tree and you want to check which one is called more often. This is what locators are useful for. If we look closely this locator looks almost like the key of a change

In [28]:
print pmc_tuple

PathSimulator
 +- RootMover
     +- Minus
         +- EnsembleFilter
             +- ConditionalSequential
                 +- MinusSubtrajectoryChooser
                 |   +- FinalSubtrajectorySelect
                 +- InterfaceSetChooser
                 +- MinusExtensionDirectionChooser
                     +- BackwardExtend


In [29]:
pm.enum[7]

(PathSimulator,
 (RootMover,
  (Minus,
   (EnsembleFilter,
    (ConditionalSequential,
     (MinusSubtrajectoryChooser, (FinalSubtrajectorySelect,)),
     (InterfaceSetChooser,),
     (MinusExtensionDirectionChooser, (BackwardExtend,)))))))

In [30]:
pm[pm.locate(medc)[0]]

MinusExtensionDirectionChooser

In [31]:
pm[1][1]

ShootingChooser

In [32]:
%%timeit
a = pm[medc_locator]

10000 loops, best of 3: 125 µs per loop


In [33]:
for pmc_tuple in st.steps[1:]:
    print ''.join([ '+' if pc[0] in p.treelogic.TupleTree(pmc_tuple.change.unique) else ' ' for pc in pm.locators().iteritems() ])

+ +    +                      +           +  
+  +   +                         +        +  
+         +                        +      +  
+    + +                    +             +  
+  +   +                         +        +  
+      +                +           +     +  
+++    +                                  +  
+                    +               +    +  
+                                  +      + +
+   +                              +      +  
+ +    +                      +           +  
+                                  +    + +  
+                   +                +    +  
+                                  +    + +  
+                                  +      + +
+      ++       +                         +  
+                 +                  +    +  
+                                  +      + +
+                                 +  +    +  
+   +                              +      +  
+++    +                                  +  
+      +                    +     

In [34]:
print p.treelogic.TupleTree.treeprint(medc_locator)
for pc in step_changes:
    if medc_locator in pc:
        print
        print p.treelogic.TupleTree.treeprint(pc.unique)

PathSimulator
 +- RootMover
     +- Minus
         +- EnsembleFilter
             +- ConditionalSequential
                 +- MinusSubtrajectoryChooser
                 +- InterfaceSetChooser
                 +- MinusExtensionDirectionChooser

PathSimulator
 +- RootMover
     +- Minus
         +- EnsembleFilter
             +- ConditionalSequential
                 +- MinusSubtrajectoryChooser
                 |   +- FinalSubtrajectorySelect
                 +- InterfaceSetChooser
                 +- MinusExtensionDirectionChooser
                     +- BackwardExtend

PathSimulator
 +- RootMover
     +- Minus
         +- EnsembleFilter
             +- ConditionalSequential
                 +- MinusSubtrajectoryChooser
                 |   +- FirstSubtrajectorySelect
                 +- InterfaceSetChooser
                 +- MinusExtensionDirectionChooser
                     +- ForwardExtend

PathSimulator
 +- RootMover
     +- Minus
         +- EnsembleFilter
             +- Condi

Now use the locator to test for specific changes

In [35]:
%%time
for pc in step_changes:
    if medc_locator in pc:
        print p.treelogic.TupleTree.treeprint(pc.unique)
        print

PathSimulator
 +- RootMover
     +- Minus
         +- EnsembleFilter
             +- ConditionalSequential
                 +- MinusSubtrajectoryChooser
                 |   +- FinalSubtrajectorySelect
                 +- InterfaceSetChooser
                 +- MinusExtensionDirectionChooser
                     +- BackwardExtend

PathSimulator
 +- RootMover
     +- Minus
         +- EnsembleFilter
             +- ConditionalSequential
                 +- MinusSubtrajectoryChooser
                 |   +- FirstSubtrajectorySelect
                 +- InterfaceSetChooser
                 +- MinusExtensionDirectionChooser
                     +- ForwardExtend

PathSimulator
 +- RootMover
     +- Minus
         +- EnsembleFilter
             +- ConditionalSequential
                 +- MinusSubtrajectoryChooser
                 |   +- FinalSubtrajectorySelect
                 +- InterfaceSetChooser
                 +- MinusExtensionDirectionChooser
                     +- BackwardExtend

CP

In [36]:
pmc = st.steps[3].change
print [(x, y.mover, p.treelogic.TupleTree(x).tail) for x, y in pmc.items()]

[((PathSimulator,), PathSimulator, PathSimulator), ((PathSimulator, (RootMover, (ReversalChooser, (PathReversal,)))), PathReversal, PathReversal), ((PathSimulator, (RootMover,)), RootMover, RootMover), ((PathSimulator, (RootMover, (ReversalChooser,))), ReversalChooser, ReversalChooser)]


In this case we get the same result as before. Note that the second method is more powerful, but also slower!

### Remember the following things about moves and changes

1. movers and changes represent a tree like structure of movers.
2. a change is a concrete realization of moves (and hence movers) while a mover itself has a certain degree of choice. 
3. Some nodes in a mover can have different outcomes, like a RandomChoiceMover will pick one of several choices. The change it generates will have only the chosen one, while the RandomChoiceMover itself has all possibilities.
4. A concrete tree like structure (without choice) of movers can be represented by a tree of tuples. Reading the tuple-tree from left to right the opening bracket '(' means go down a level (a new child) while the closing bracket means trael up a level ')' go to the parent.
5. The smallest unique tupletree for a pmc can be accessed using `.unique`

In [36]:
first_minus_change = filter(lambda x : p.MinusMover in x, step_changes)[0]

In [37]:
print first_minus_change.unique

(PathSimulator, (RootMover, (Minus, (EnsembleFilter, (ConditionalSequential, (MinusSubtrajectoryChooser, (FinalSubtrajectorySelect,)), (InterfaceSetChooser,), (MinusExtensionDirectionChooser, (BackwardExtend,)))))))


will be interpreted as

In [38]:
print p.treelogic.TupleTree.treeprint(first_minus_change.unique)

PathSimulator
 +- RootMover
     +- Minus
         +- EnsembleFilter
             +- ConditionalSequential
                 +- MinusSubtrajectoryChooser
                 |   +- FinalSubtrajectorySelect
                 +- InterfaceSetChooser
                 +- MinusExtensionDirectionChooser
                     +- BackwardExtend


In [39]:
%%timeit
pm.enum

1000 loops, best of 3: 259 µs per loop


In [40]:
%%timeit
pmc in pm

1000 loops, best of 3: 214 µs per loop


In [42]:
%%timeit
[p in pmc for p in pm.enum]

1000 loops, best of 3: 872 µs per loop


In [43]:
%%timeit
pm in pm

The slowest run took 7.20 times longer than the fastest. This could mean that an intermediate result is being cached 
100000 loops, best of 3: 2.65 µs per loop


In [44]:
%%timeit
pmc.unique in pm

1000 loops, best of 3: 225 µs per loop
